In [1]:
import numpy as np
from numpy.linalg import inv
from numpy.linalg import det
from scipy.optimize import minimize
from matplotlib import pyplot as plt
%matplotlib inline

In [13]:
# Definition of all the functions are here

def calculate_rate_from_prices(P):
    rate = np.zeros((P.shape[0]-1, P.shape[1]))
    for i in range(rate.shape[0]):
        for j in range(rate.shape[1]):
            rate[i][j] = P[i+1][j]/P[i][j] - 1
    return rate

def markowitz_optimization_model(OM, M, R, n, m, number=1):
    
    if number == 1:
        I = np.ones((m,1))
        ans = np.dot(np.linalg.inv(OM), I)
        ans /= np.dot(np.dot(I.T, np.linalg.inv(OM)), I)
        return ans.T
    
    if number == 2:
        e = np.ones((m, 1))
        numerator = np.array([[2*R, float(M.T @ inv(OM) @ e)], [2, float(e.T @ inv(OM) @ e)]])
        denominator = np.array([[float(M.T @ inv(OM) @ M), float(M.T @ inv(OM) @ e)],
                                [float(e.T @ inv(OM) @ M), float(e.T @ inv(OM) @ e)]])
        lambda1 = det(numerator)/det(denominator)
        numerator = np.array([[float(M.T @ inv(OM) @ M), 2*R], 
                                [float(e.T @ inv(OM) @ M), 2]])
        denominator = np.array([[float(M.T @ inv(OM) @ M), float(M.T @ inv(OM) @ e)],
                                [float(e.T @ inv(OM) @ M), float(e.T @ inv(OM) @ e)]])

        lambda2 = det(numerator)/det(denominator)
        sol = 0.5 * (lambda1 * (inv(OM) @ M) + lambda2 * (inv(OM) @ e.reshape(m)))
        return sol
    
    if number == 3:
        Objective = lambda w: w.T @ OM @ w
        def equality(x):
            e = np.ones((m))
            return(e @ x - 1)
        def inequality(x):
            return(M @ x - 0.3)

        b = (-np.inf, np.inf)
        temp=[]
        for i in range(m):
            temp.append(b)
        limit = tuple(temp)
        y = np.ones(m)
        constraint1 = {'type':'ineq', 'fun':inequality}
        constraint2 = {'type':'eq', 'fun':equality}
        constraints = ([constraint1, constraint2])
        sol = minimize(Objective, y, method='SLSQP', \
                            bounds=limit, constraints=constraints)
        return sol



In [12]:
import numpy as np


P = [
    [10, 15, 10], 
    [15, 20, 15],
    [13, 18, 11],
    [15, 15, 14],
    [14, 16, 13]
]

P = np.array(P)

S = calculate_rate_from_prices(P)
n, m = S.shape

M = np.mean(S, axis=0)
DM = S - M
OM = np.dot(DM.T, DM) * 1/n

print("1.a] Covariance matrix of return: ")
print(np.linalg.inv(OM))


1.a] Covariance matrix of return: 
[[ 505.40144006 -157.391795   -347.41202632]
 [-157.391795     88.20495343   93.92524336]
 [-347.41202632   93.92524336  255.32288321]]


In [14]:
markowitz_optimization_model(OM, M, 0, n, m)

array([[0.02199382, 0.91043323, 0.06757295]])

In [15]:
markowitz_optimization_model(OM, M, 0.3, n, m, number=2)

array([ 7.48581885, -2.06285174, -4.42296711])

In [16]:
markowitz_optimization_model(OM, M, 0.3, n, m, number=3)

     fun: 0.17218315452876504
     jac: array([0.14996823, 0.06646391, 0.14496227])
 message: 'Optimization terminated successfully'
    nfev: 24
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([ 7.48581982, -2.06285163, -4.42296819])

In [17]:
# Let R_1 = -1, R_2 = 1
A = np.array([
    [10, 15, 10], 
    [15, 20, 15],
    [13, 18, 11],
    [15, 15, 14],
    [14, 16, 13]
])
R_1 = -1.0
R_2 = 1.0
A = A.T[:2].T

rate = np.zeros((A.shape[0]-1, A.shape[1]))
for i in range(rate.shape[0]):
    for j in range(rate.shape[1]):
        rate[i][j] = A[i+1][j]/A[i][j] - 1

n, m = rate.shape
mu = np.mean(rate, axis=0)
D = rate - mu
# Covariance matrix of return
om = (D.T @ D) *1/n
e = np.ones((m, 1))
w_1 = markowitz_model2(om, mu, R_1)
w_2 = markowitz_model2(om, mu, R_2)

n = 100
l = np.linspace(0, 1, n)
ret = np.zeros(n)
risk = np.zeros(n) #[0]*n
risk1 = w_1.T@om@w_1 
risk2 = w_2.T@om@w_2

for i in range(n):
    ret[i] = l[i]*R_1 + (1 - l[i])*R_2
    risk[i] =  ((risk1 * l[i]**2) + (risk2 * (1 - l[i])**2))**0.5

#Calculating minimum variance points    
risk_model1 = risk[np.argmin(risk)]
ret_model1 = ret[np.argmin(risk)]
index = np.where(np.isclose(ret,0.3, 1e-1))[0]
risk_model2 = risk[index]

#Plotting the data on a graph
print("1)c) The efficient frontier with the minimum variance point for Markowitz model 1 and 2 is as follows: ")
fig = plt.plot(risk, ret, '-')
plt.xlabel('Risk')
plt.ylabel('Return')
plt.plot(risk_model1, ret_model1, 'ro')
plt.annotate("  Model I minimum variance", (risk_model1, ret_model1))
plt.plot(risk_model2[1],0.3, 'ro')
plt.annotate("    Model II minimum variance", (risk_model2[1], 0.3))
plt.show()

NameError: name 'markowitz_model2' is not defined